In [1]:
# COMMANDS 3-7: Functions for generating content based filtering reccomendations
# COMMANDS 9-18: Recommendations based on attributes
# COMMAND 9: Load restaurant attribute file
# COMMAND 10: Convert to Pandas DataFrame and change True False entries to 1 and 0 entries
# COMMAND 11: filter down to designated city, change columns to int data types where needed
# city = 'Las Vegas'
# COMMAND 12: Generate dictionary with all restaurant information
# COMMAND 13: Write dictionary to json file
# COMMAND 14: drop useless columns
# COMMAND 15: convert pandas dataframe to array and get similarity matrix
# COMMAND 16: Construct a reverse map of indices and movie titles
# COMMAND 17-18: Generate reccomendations

# COMMANDS 20-22: Recommendations based on text reviews
# COMMAND 20-22: executes the same reccomendations except using text data 

In [2]:
# import statements
import numpy as np
import pandas as pd
import string
from collections import Counter
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
import json

In [3]:
##############################################
#### FUNCTION FOR LOADING BIG DATA AS TSV ####
##############################################

def loadData(file_location, delimiter = "\t"):
  file_type = "csv"

  # CSV options
  infer_schema = "false"
  first_row_is_header = "true"
#   delimiter = "\t"

  # The applied options are for CSV files. For other file types, these will be ignored.
  df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .option("header", first_row_is_header) \
    .option("sep", delimiter) \
    .load(file_location)
  return df

In [4]:
# Function that takes in a list of restaurants as input and outputs most similar restaurants
def get_recommendations(title, cosine_sim, restDF):
    sim_scores = [(0, 0)]*cosine_sim.shape[0]
    for t in title:
      # Get the index of the movie that matches the title
      print("evaluating restaurant", t)
      idx = indices[t]
      print("scoring")
      # Get the pairwsie similarity scores of all movies with that movie
      scores = list(enumerate(cosine_sim[idx]))
      print("aggregating scores")
      sim_scores = [(i, scores[i][1] + sim_scores[i][1]) for i in range(cosine_sim.shape[0])]
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]
    
    # Get the movie indices
    print("retrieving indices")
    movie_indices = [i[0] for i in sim_scores]
    print("returning value")
    # Return the top 10 most similar movies
    return restDF['business_id'].iloc[movie_indices]
  

In [5]:
# function that converts the ids to names
def idToName(recs):
  alist = []
  for i in recs:
    print(str(restaurants[i]["stars"]) + " " + restaurants[i]["name"])
    alist.append([(restaurants[i]["stars"]), restaurants[i]["name"]])
  return alist

In [6]:
## get the data

def gatherCommentsRestaurant(df, a, b):
  # aggregate all restaurant comments
  restComments = {}
  
  for row in df.rdd.toLocalIterator():
    if a == b:
      break
    a += 1
    rest = row['business_id']
    if rest not in restComments:
      restComments[rest] = row['text']
    else:
      restComments[rest] += ' ' + row['text']
  
  # make restaurant | text dataframe
  business_id = []
  text = []
  itt = a
  
  restDF = {}
  
  for key, value in restComments.items():
    business_id.append(key)
    text.append(value)
    
  restDF = {'business_id': business_id,
             'text': text}

  restComments = pd.DataFrame(restDF,columns= ['business_id', 'text'])
  return restComments


In [7]:
def commentsToAttributes(restDF):
  #Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
  tfidf = TfidfVectorizer(stop_words='english')

  #Replace NaN with an empty string
  restDF['text'] = restDF['text'].fillna('')

  #Construct the required TF-IDF matrix by fitting and transforming the data
  tfidf_matrix = tfidf.fit_transform(restDF['text'])

  return tfidf_matrix

In [8]:
########################################
## FILTERING BY RESTAURANT ATTRIBUTES ##
########################################

In [9]:
#######################################
## IMPORT RESTAURANT ATTRIBUTES FILE ##
#######################################

df_attr = loadData("/FileStore/tables/dFile2.tsv")
display(df_attr)

name,business_id,address,city,stars,Boating,Supper Clubs,Bakeries,Event Planning & Services,Private Tutors,Cajun/Creole,Belgian,Guamanian,Hotels & Travel,Szechuan,Fire Protection Services,Mosques,Nightlife,Animal Physical Therapy,Himalayan/Nepalese,Auto Detailing,Buffets,Wine & Spirits,Body Shops,Public Markets,Paint & Sip,Kids Activities,Indonesian,Musical Instruments & Teachers,Bubble Tea,Hainan,Real Estate,Pumpkin Patches,Skin Care,Dim Sum,Engraving,Tuscan,Yoga,Laser Tag,South African,Tours,Visitor Centers,Appliances,General Dentistry,Sporting Goods,Wigs,Pawn Shops,WiFi,Bankruptcy Law,Home Cleaning,Antiques,romantic,BYOB,Estate Planning Law,Hostels,Izakaya,Investing,Recording & Rehearsal Studios,Unofficial Yelp Events,Southern,Eatertainment,Ice Delivery,Archery,Audio/Visual Equipment Rental,Pita,Plus Size Fashion,Wine Bars,Specialty Schools,Caterers,Cabaret,Grilling Equipment,Head Shops,Whiskey Bars,British,Furniture Stores,Mattresses,dinner,Vacation Rentals,Amateur Sports Teams,Used Bookstore,Bistros,Caters,Senegalese,Hair Salons,Auto Glass Services,Music Venues,Gelato,Siding,Cannabis Collective,Sushi Bars,wednesday,Souvenir Shops,African,Glass & Mirrors,Medical Cannabis Referrals,Food,Heating & Air Conditioning/HVAC,Funeral Services & Cemeteries,Screen Printing,Windshield Installation & Repair,Shaved Snow,Hiking,africanamerican,Gutter Services,Medical Centers,Limos,Bar Crawl,Grocery,Door Sales/Installation,lunch,Counseling & Mental Health,Zoos,Nail Technicians,Patisserie/Cake Shop,Comfort Food,Mongolian,Herbs & Spices,Community Service/Non-Profit,Ethnic Grocery,Colleges & Universities,Custom Cakes,Dentists,Pressure Washers,Newspapers & Magazines,Thai,Filipino,Commercial Real Estate,American (Traditional),Marinas,Tex-Mex,Cupcakes,Massage,Chocolatiers & Shops,Men's Clothing,garage,AcceptsInsurance,Business Consulting,Butcher,Transportation,Signmaking,RestaurantsTableService,friday,Veterinarians,Hotel bar,breakfast,Popcorn Shops,Social Clubs,Tax Law,Historical Tours,Soccer,Computers,Art Museums,Honey,Discount Store,Auto Repair,Formal Wear,Art Schools,Motorcycle Repair,Pool & Billiards,Baby Gear & Furniture,Eyebrow Services,Modern European,Mailbox Centers,Metal Fabricators,Tonkatsu,Makeup Artists,Themed Cafes,dj,Vape Shops,Vegetarian169,Windows Installation,Employment Agencies,Cafes,Fruits & Veggies,Personal Chefs,Indoor Playcentre,Gay Bars,Churches,Tempura,Beer Bar,tuesday,Furniture Repair,RV Repair,American (New),Country Dance Halls,Vegan185,Health & Medical,Breakfast & Brunch,Wine Tours,Local Fish Stores,Waxing,Beer Hall,Pick Your Own Farms,Real Estate Services,Meditation Centers,Personal Shopping,Bike Repair/Maintenance,Caribbean,Party Bus Rentals,Shopping,Alternative Medicine,Home Window Tinting,kids,Middle Schools & High Schools,Parks,Towing,Graphic Design,Carpet Installation,Party & Event Planning,Tanning,Art Supplies,Stadiums & Arenas,Holiday Decorations,Chicken Wings,Special Education,Fitness & Instruction,Cosmetics & Beauty Supply,Marketing,Aircraft Repairs,BYOBCorkage,Video Game Stores,Bocce Ball,video,Hair Removal,Champagne Bars,Wholesale Stores,Advertising,classy,Mortgage Brokers,Shared Office Spaces,Cafeteria,Plumbing,Diagnostic Services,Hot Pot,Couriers & Delivery Services,Lakes,Horse Racing,Golf Equipment Shops,Traditional Clothing,Corkage,RestaurantsReservations,Dance Studios,Ice Cream & Frozen Yogurt,Landscape Architects,Pets,Banks & Credit Unions,Comic Books,Occupational Therapy,Gastropubs,Pasta Shops,Cooking Classes,Trinidadian,Coffee & Tea Supplies,Currency Exchange,saturday,Bangladeshi,Arabian,Spray Tanning,Trusts,RestaurantsTakeOut,Strip Clubs,brunch,coloring,extensions,Bookstores,Donairs,Personal Injury Law,RestaurantsCounterService,Printing Services,Soba,Afghan,Nicaraguan,BusinessAcceptsBitcoin,Cannabis Clinics,Game Meat,Dance Clubs,Japanese Curry,WheelchairAccessible,Keys & Locksmiths,Martial Arts,Lounges,Beauty & Spas,Bartenders,Education,Piano Bars,Australian,Outdoor Gear,Irish,Gas Stations,Naturop

In [10]:
##########################
###### CLEAN FILE ########
##########################
print("Converting to Pandas")
df_attr = df_attr.toPandas()

print("Replacing Values: True")
df_attr = df_attr.replace(to_replace ="True", 
                 value ="1")
print("Replacing Values: False")
df_attr = df_attr.replace(to_replace ="False", 
                 value ="0")


Converting to Pandas
Replacing Values: True
Replacing Values: False

In [11]:
#filtering city
print("filtering down to city")
filtered = df_attr[df_attr.city == city]
filtered.reset_index(drop=True, inplace=True)

#concert numbers to int data type
columns = list(filtered.columns)
columns.remove('name')
columns.remove('business_id')
columns.remove('address')
columns.remove('city')
print("converting to numeric types")
i = 0
for col in columns:
  if i%50 == 0:
    print("converting", str(i) + "th column")
  i += 1
  filtered[col] = pd.to_numeric(filtered[col],errors='coerce')
filtered
  

,name,business_id,address,city,stars,Boating,Supper Clubs,Bakeries,Event Planning & Services,Private Tutors,...,Officiants,Security Services,Police Departments,Transmission Repair,Tea Rooms,Pet Services,Books,Mobile Phone Repair,Mauritius,Orthodontists
0,Carluccio's Tivoli Gardens,PZ-LZzSlhSe9utkQYU8pFg,"1775 E Tropicana Ave, Ste 29",Las Vegas,4.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Maria's Mexican Restaurant & Bakery,tstimHoMcYbkSC4eBA1wEg,6055 E Lake Mead Blvd,Las Vegas,4.5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Dairy Queen,kANF0dbeoW34s2vwh6Umfw,6125 Spring Mountain Rd,Las Vegas,2.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Artisan Fine Dining Room,bJP4l_BGq2CudEu0m-wNjg,"Artisan Hotel, 1501 W Sahara Ave",Las Vegas,2.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bistro Divino,v7ZFEEqJBg_r8NCwHRP_2A,241 W Charleston Blvd,Las Vegas,4.5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,La Creperie,6fPQJq4f_yiq1NHn0fd11Q,3655 Las Vegas Blvd S,Las Vegas,3.5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Feast Buffet,k-dDZvTeLysoJvjHI-qr9g,2411 W Sahara Ave,Las Vegas,3.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Stage Deli Of Las Vegas,PWjgwphd2prhtqTPWwLbXw,"3500 Las Vegas Blvd S, Ste E11",Las Vegas,3.5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Pita Pit,eY98EH_PHGnJVaJ7t2UpoQ,1263 Silverado Ranch Blvd,Las Vegas,4.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Mr G's Pub & Grub,iwGhazq9eP51PSerTrMrwg,"3342 E Sandhill Rd, Ste 11",Las Vegas,4.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
########################################################
### RESTAURANT DICTIONARY TO GET INFO ON RESTAURANTS ###
########################################################

print("Generating restaurant ID dictionary to save restaurant [name, address, city]")
restaurants = {}
for index, row in df_attr.iterrows():
  restaurants[row['business_id']] = {'name': row['name'], 'address': row['address'], 'city': row['city'], 'stars': row['stars']}
  

Generating restaurant ID dictionary to save restaurant [name, address, city]

In [13]:
# save dictionary as json
import json

# access file at
# https://community.cloud.databricks.com/dbfs/tmp/restaurantAttrDict.json
json = json.dumps(str(restaurants))
f = open("/dbfs/tmp/restaurantAttrDict.json","w")
f.write(json)
f.close()

In [14]:
print("droping columns: name, address, city, business_id")
filtered2 = filtered.drop('name', axis=1)
filtered2 = filtered2.drop('address', axis = 1)
filtered2 = filtered2.drop('city', axis=1)
filtered2 = filtered2.drop('business_id', axis=1)

droping columns: name, address, city, business_id

In [15]:
# convert pandas dataframe to numpy
print("converting dataframe to np_array")
filtered2 = filtered2.values

# Compute the cosine similarity matrix
print("computing similarity matrix")
cosine_sim = linear_kernel(filtered2, filtered2)


converting dataframe to np_array
computing similarity matrix

In [16]:
#Construct a reverse map of indices and movie titles
print("Constructing a reverse map of indices and restaurant titles")
# filtered.reset_index(drop=True, inplace=True)
indices = pd.Series(filtered.index, index=filtered['business_id']).drop_duplicates()


Constructing a reverse map of indices and restaurant titles

In [17]:
print("generating recomendations")
recs = get_recommendations(['ngs16C2M_uTq2zXamltHVw'], cosine_sim, filtered)
print(recs)

generating recomendations
evaluating restaurant ngs16C2M_uTq2zXamltHVw
scoring
aggregating scores
retrieving indices
returning value
1164 H8FovVGJV9Uut-BYFPeOCg
1447 9B9rmmyvI7uWx9C_BFU59A
1781 0n3vcq00Z4ijqwwn-INfkg
2259 kcxD07NmyWxAHvrzS7LD8A
2802 hfgcYtGziLl5bGbc89BTkQ
5685 CmzJMXgsD7deXT7QSaLi1Q
6283 kOo4ZY2UQAX4j312mzQ8mA
6293 ngs16C2M_uTq2zXamltHVw
167 sgtF_x5ppBbIXRUE8XrmVg
264 6LkNSA3b3dufG5_dKfEQdQ
Name: business_id, dtype: object

In [18]:
idToName(recs)

5.0 Jamaican Joint Catering
5.0 Smokeit BBQ & Catering
5.0 Can't Dutch This
5.0 Love Empanadas
5.0 Geo's tacos
5.0 BackYard BBQ
5.0 Taco Naco
4.5 Jessie Rae's BBQ
5.0 Sticky Iggy's
5.0 Southern Kitchen
 Out[ 42 ]: [['5.0', 'Jamaican Joint Catering'],
 ['5.0', 'Smokeit BBQ & Catering'],
 ['5.0', "Can't Dutch This"],
 ['5.0', 'Love Empanadas'],
 ['5.0', "Geo's tacos"],
 ['5.0', 'BackYard BBQ'],
 ['5.0', 'Taco Naco'],
 ['4.5', "Jessie Rae's BBQ"],
 ['5.0', "Sticky Iggy's"],
 ['5.0', 'Southern Kitchen']]

In [19]:
#######################################
## FILTERING BY RESTAURANT COMMENTS ###
#######################################

In [20]:
######################################
## IMPORT RESTAURANT COMMENTS FILE ###
######################################

# File location and type
file_location = "/FileStore/tables/dFile1.tsv"

# load data
df = loadData(file_location)

display(df)

user_id,business_id,text,stars
dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,"Went in for a lunch. Steak sandwich was delicious, and the Caesar salad had an absolutely delicious dressing, with a perfect amount of dressing, and distributed perfectly across each leaf. I know I'm going on about the salad ... But it was perfect.Drink prices were pretty good.The Server, Dawn, was friendly and accommodating. Very happy with her.In summation, a great pub experience. Would go again!",5.0
w31MKYsNFMrjhWxxAb5wIw,eU_713ec6fTGNO4BegRaww,"""I'll be the first to admit that I was not excited about going to La Tavolta. Being a food snob, when a group of friends suggested we go for dinner I looked online at the menu and to me there was nothing special and it seemed overpriced. Im also not big on ordering pasta when I go out. Alas, I was outnumbered. Thank goodness! I ordered the sea bass special. It was to die for. Cooked perfectly, seasoned perfectly, perfect portion. I can not say enough good things about this dish. When the server asked how it was he seemed very proud of the dish and said, """" doesn't she (the chef) do an incredible job?"""" She does. My hubby got the crab tortellini and also loved his. I heard """"mmmm this is so good"""" from all around the table. Our waiter was super nice and even gave us free desserts because we were some of the last people in the restaurant. Service was very slow and the place was PACKED but we had our jugs of wine and a large group with good conversation so it didn't seem to bother anyone.So-Do order the calamari and fried zucchini appetizers. Leave out the mussels. If they have the sea bass special, I highly recommend it. The chicken parm and crab tortellini were also very good and very big. The chicken Romano was a bit bland. The house salads were teeny. Do make a reservation but still expect to wait for your food. Go with a large group of people and plan for it to be loud. Don't go with a date unless you're fighting and don't feel like hearing anything they have to say. Ask to sit in the side room if it's available.""",4.0
jlu4CztcSxrKx56ba1a5AQ,3fw2X5bZYeW9xCz_zGhOHg,"Tracy dessert had a big name in Hong Kong and the one in First Markham place has been here for many years now! Came in for some Chinese dessert, and I must say their selection has increased tremendously over the years. I might as well add that the price has also increased tremendously as well. The waitress gave us tea, which I could taste had red date in it. Fancy!A simple taro with coconut with tapioca pearls was like $5.25 or something. Basically all the desserts were more than $5. That's crazy! I can literally just make this dessert at home and for a bowl, it would probably cost like $0.50. A few years ago, I think I can still get it for like $3-$4, which is more reasonable, but wow, more than $5 is a little over the top for this dessert. Though I must say, it is Tracy Dessert, and they are a little more on the expensive side. I also saw other items on the menu like fish balls, chicken wings, shaved ice. My friend got a mango drink with fresh mango in it! I'm also surprised how many people come to Tracy Dessert after work. We came on a Sunday and the tables were always filled. I think the amount of tables they had were just perfect because no one really waited for seats for a long time, but the tables kept filling up once a table was finished.",3.0
d6xvYpyzcfbF_AZ8vMB7QA,zvO-PJCpNk4fgAVUnExYAA,"This place has gone down hill. Clearly they have cut back on staff and food qualityMany of the reviews were written before the menu changed. I've been going for years and the food quality has gone down hill.The service is slow & my salad, which was $15, was as bad as it gets.It's just not worth spending the money on this place when there are so many other options.",1.0
FIk4lQQu1eTe2EpzQ4xhBA,8mIrX_LrOnAqWsB5JrOojQ,"""Like walking back in time, every Saturday morning my sister and I was in a bowling league and after we were done, we'd spend

In [21]:
# aggregate comments for each restaurant
restDF = gatherCommentsRestaurant(df,0, 100)

# convert comments to attributes
tfidf_matrix = commentsToAttributes(restDF)

# Compute the cosine similarity matrix
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

#Construct a reverse map of indices and movie titles
indices = pd.Series(restDF.index, index=restDF['business_id']).drop_duplicates()

recommendations = get_recommendations(['eU_713ec6fTGNO4BegRaww'], cosine_sim, restDF)

recommendations

evaluating restaurant eU_713ec6fTGNO4BegRaww
scoring
aggregating scores
retrieving indices
returning value
 Out[ 45 ]: 28 cHdJXLlKNWixBXpDwEGb_A
7 N0apJkxIem2E8irTBRKnHw
33 Sfc8Haz2Yri8Mo1L0E0TsA
26 xHZmM-kGXqOfmQNTm0T05g
81 YSUcHqlKMPHHJ_cTrqtNrA
84 0xyqLvtr0Zbr4G-gML-QmQ
15 Gyrez6K8f1AyR7dzW9fvAw
60 HhVmDybpU7L50Kb5A0jXTg
52 5qG4UHurI1yEozwn25WAFw
12 9mIm1ef-NVDQHFE39Faxxg
Name: business_id, dtype: object

In [22]:
idToName(recommendations)

4.0 Chino Bandido
4.0 Wolfgang Puck Bar & Grill Las Vegas
3.0 Chicken Bonz
4.0 Circles Grill and Market
3.0 Vagabondo Italian Ristorante+Lounge
3.5 Fleming's Prime Steakhouse & Wine Bar- Chandler
4.0 Casanova
3.5 Guy Fieri's Vegas Kitchen & Bar
4.0 BLACKOUT Dining in the Dark
3.5 Vine Tavern & Eatery
 Out[ 46 ]: [['4.0', 'Chino Bandido'],
 ['4.0', 'Wolfgang Puck Bar & Grill Las Vegas'],
 ['3.0', 'Chicken Bonz'],
 ['4.0', 'Circles Grill and Market'],
 ['3.0', 'Vagabondo Italian Ristorante+Lounge'],
 ['3.5', "Fleming's Prime Steakhouse & Wine Bar- Chandler"],
 ['4.0', 'Casanova'],
 ['3.5', "Guy Fieri's Vegas Kitchen & Bar"],
 ['4.0', 'BLACKOUT Dining in the Dark'],
 ['3.5', 'Vine Tavern & Eatery']]